### Closing Generators

Consider this generator function

In [1]:
def read_file(f_name):
    f = open(f_name)
    try:
        for row in f:
            yield row
    finally:
        f.close()

Suppose the file has 100 rows

In [ ]:
rows = read_file('test.txt')
for _ in range(10):
    next(rows)

- Have now read 10 rows
- But the file is still open
- So how do we close the file without iterating through the entire file?

#### Closing a generator

We have seen the possible generator states: **created**, **running**, **suspended**, **closed**

We can close a generator by calling its close() method

In [ ]:
def read_file(f_name):
    f = open(f_name)
    try:
        for row in f:
            yield row
    finally:
        f.close()
        
rows = read_file('test.txt')
for _ in range(10):
    next(rows)
    
rows.close() # finally block runs, and file is closed

#### Behind the scenes:

When .close() is called, an exception is triggered inside the generator  
The exception is a *GeneratorExit* exception

In [6]:
def gen():
    try:
        yield 1
        yield 2
    except GeneratorExit:
        print('Generator close called')
    finally:
        print('Cleanup here...')

In [7]:
g = gen()

In [8]:
next(g)

1

In [9]:
g.close()

Generator close called
Cleanup here...


Python has certain expectations when close() is called
- a GeneratorExit exception bubbles up -> the exception is silenced by Python
- the generator exits cleanly (returns) -> to the caller, everything works 'normally'
- some other exception is raised from inside the generator -> exception is seen by caller

However, you cannot just ignore a GeneratorExit exception. If the generator "ignores" the GeneratorExit exception and yields another value -> Python raises a RuntimeError: generator ignored GeneratorExit

In other words, don't try to catch and ignore a GeneratorExit exception

It's perfectly OK not the catch it and simply let it bubble up

In [10]:
def gen():
    yield 1
    yield 2

g = gen()
next(g)
g.close()

No issues seen!

#### Use in coroutines

Since coroutine are generator functions, it is OK to close a coroutine also

For example, you may have a coroutine that receives data to write to a database
- coroutine opens a transaction when it is primed(next)
- coroutine receives data to write to the database
- can now do one of two things:
- 1. coroutine commits the transaction when close() is called (GeneratorExit)
- 2. coroutine aborts (roll back) transaction if some other exception occurs

#### Code Examples

In [11]:
from inspect import getgeneratorstate

In [12]:
import csv

def parse_file(f_name):
    print('opening file...')
    f = open(f_name, 'r')
    try:
        dialect = csv.Sniffer().sniff(f.read(2000))
        f.seek(0)
        reader = csv.reader(f, dialect=dialect)
        for row in reader:
            yield row
    finally:
        print('closing file')
        f.close()

In [13]:
import itertools

parser = parse_file('cars.csv')
for row in itertools.islice(parser, 10):
    print(row)

opening file...
['Car', 'MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model', 'Origin']
['Chevrolet Chevelle Malibu', '18.0', '8', '307.0', '130.0', '3504.', '12.0', '70', 'US']
['Buick Skylark 320', '15.0', '8', '350.0', '165.0', '3693.', '11.5', '70', 'US']
['Plymouth Satellite', '18.0', '8', '318.0', '150.0', '3436.', '11.0', '70', 'US']
['AMC Rebel SST', '16.0', '8', '304.0', '150.0', '3433.', '12.0', '70', 'US']
['Ford Torino', '17.0', '8', '302.0', '140.0', '3449.', '10.5', '70', 'US']
['Ford Galaxie 500', '15.0', '8', '429.0', '198.0', '4341.', '10.0', '70', 'US']
['Chevrolet Impala', '14.0', '8', '454.0', '220.0', '4354.', '9.0', '70', 'US']
['Plymouth Fury iii', '14.0', '8', '440.0', '215.0', '4312.', '8.5', '70', 'US']
['Pontiac Catalina', '14.0', '8', '455.0', '225.0', '4425.', '10.0', '70', 'US']


In [14]:
parser.close()

closing file


In [15]:
def parse_file(f_name):
    print('opening file...')
    f = open(f_name, 'r')
    try:
        dialect = csv.Sniffer().sniff(f.read(2000))
        f.seek(0)
        reader = csv.reader(f, dialect=dialect)
        for row in reader:
            yield row
    except Exception as ex:
        print('some exception occured', str(ex))
    except GeneratorExit:
        print('Generator was closed')
    finally:
        print('closing file')
        f.close()

In [17]:
parser = parse_file('cars.csv')
for row in itertools.islice(parser, 10):
    print(row)

Generator was closed
closing file
opening file...
['Car', 'MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model', 'Origin']
['Chevrolet Chevelle Malibu', '18.0', '8', '307.0', '130.0', '3504.', '12.0', '70', 'US']
['Buick Skylark 320', '15.0', '8', '350.0', '165.0', '3693.', '11.5', '70', 'US']
['Plymouth Satellite', '18.0', '8', '318.0', '150.0', '3436.', '11.0', '70', 'US']
['AMC Rebel SST', '16.0', '8', '304.0', '150.0', '3433.', '12.0', '70', 'US']
['Ford Torino', '17.0', '8', '302.0', '140.0', '3449.', '10.5', '70', 'US']
['Ford Galaxie 500', '15.0', '8', '429.0', '198.0', '4341.', '10.0', '70', 'US']
['Chevrolet Impala', '14.0', '8', '454.0', '220.0', '4354.', '9.0', '70', 'US']
['Plymouth Fury iii', '14.0', '8', '440.0', '215.0', '4312.', '8.5', '70', 'US']
['Pontiac Catalina', '14.0', '8', '455.0', '225.0', '4425.', '10.0', '70', 'US']


In [18]:
parser.close()

Generator was closed
closing file


In [19]:
def parse_file(f_name):
    print('opening file...')
    f = open(f_name, 'r')
    try:
        dialect = csv.Sniffer().sniff(f.read(2000))
        f.seek(0)
        next(f) # skip header row
        reader = csv.reader(f, dialect=dialect)
        for row in reader:
            try:
                yield row
            except GeneratorExit:
                print('ignoring call to close generator...')
    finally:
        print('closing file')
        f.close()

In [20]:
parser = parse_file('cars.csv')
for row in itertools.islice(parser, 10):
    print(row)

opening file...
['Chevrolet Chevelle Malibu', '18.0', '8', '307.0', '130.0', '3504.', '12.0', '70', 'US']
['Buick Skylark 320', '15.0', '8', '350.0', '165.0', '3693.', '11.5', '70', 'US']
['Plymouth Satellite', '18.0', '8', '318.0', '150.0', '3436.', '11.0', '70', 'US']
['AMC Rebel SST', '16.0', '8', '304.0', '150.0', '3433.', '12.0', '70', 'US']
['Ford Torino', '17.0', '8', '302.0', '140.0', '3449.', '10.5', '70', 'US']
['Ford Galaxie 500', '15.0', '8', '429.0', '198.0', '4341.', '10.0', '70', 'US']
['Chevrolet Impala', '14.0', '8', '454.0', '220.0', '4354.', '9.0', '70', 'US']
['Plymouth Fury iii', '14.0', '8', '440.0', '215.0', '4312.', '8.5', '70', 'US']
['Pontiac Catalina', '14.0', '8', '455.0', '225.0', '4425.', '10.0', '70', 'US']
['AMC Ambassador DPL', '15.0', '8', '390.0', '190.0', '3850.', '8.5', '70', 'US']


In [21]:
parser.close()

ignoring call to close generator...


RuntimeError: generator ignored GeneratorExit

In [22]:
def parse_file(f_name):
    print('opening file...')
    f = open(f_name, 'r')
    try:
        dialect = csv.Sniffer().sniff(f.read(2000))
        f.seek(0)
        next(f) # skip header row
        reader = csv.reader(f, dialect=dialect)
        for row in reader:
            try:
                yield row
            except GeneratorExit:
                print('Got call to close generator')
                raise
    finally:
        print('closing file')
        f.close()

In [23]:
parser = parse_file('cars.csv')
for row in itertools.islice(parser, 10):
    print(row)

ignoring call to close generator...
opening file...
['Chevrolet Chevelle Malibu', '18.0', '8', '307.0', '130.0', '3504.', '12.0', '70', 'US']
['Buick Skylark 320', '15.0', '8', '350.0', '165.0', '3693.', '11.5', '70', 'US']
['Plymouth Satellite', '18.0', '8', '318.0', '150.0', '3436.', '11.0', '70', 'US']
['AMC Rebel SST', '16.0', '8', '304.0', '150.0', '3433.', '12.0', '70', 'US']
['Ford Torino', '17.0', '8', '302.0', '140.0', '3449.', '10.5', '70', 'US']
['Ford Galaxie 500', '15.0', '8', '429.0', '198.0', '4341.', '10.0', '70', 'US']
['Chevrolet Impala', '14.0', '8', '454.0', '220.0', '4354.', '9.0', '70', 'US']
['Plymouth Fury iii', '14.0', '8', '440.0', '215.0', '4312.', '8.5', '70', 'US']
['Pontiac Catalina', '14.0', '8', '455.0', '225.0', '4425.', '10.0', '70', 'US']
['AMC Ambassador DPL', '15.0', '8', '390.0', '190.0', '3850.', '8.5', '70', 'US']


Exception ignored in: <generator object parse_file at 0x0000020A0E3CF048>
RuntimeError: generator ignored GeneratorExit


In [24]:
parser.close()

Got call to close generator
closing file


In [25]:
def parse_file(f_name):
    print('opening file...')
    f = open(f_name, 'r')
    try:
        dialect = csv.Sniffer().sniff(f.read(2000))
        f.seek(0)
        next(f) # skip header row
        reader = csv.reader(f, dialect=dialect)
        for row in reader:
            try:
                yield row
            except GeneratorExit:
                print('Got call to close generator')
                return
    finally:
        print('closing file')
        f.close()

In [26]:
parser = parse_file('cars.csv')
for row in itertools.islice(parser, 10):
    print(row)

opening file...
['Chevrolet Chevelle Malibu', '18.0', '8', '307.0', '130.0', '3504.', '12.0', '70', 'US']
['Buick Skylark 320', '15.0', '8', '350.0', '165.0', '3693.', '11.5', '70', 'US']
['Plymouth Satellite', '18.0', '8', '318.0', '150.0', '3436.', '11.0', '70', 'US']
['AMC Rebel SST', '16.0', '8', '304.0', '150.0', '3433.', '12.0', '70', 'US']
['Ford Torino', '17.0', '8', '302.0', '140.0', '3449.', '10.5', '70', 'US']
['Ford Galaxie 500', '15.0', '8', '429.0', '198.0', '4341.', '10.0', '70', 'US']
['Chevrolet Impala', '14.0', '8', '454.0', '220.0', '4354.', '9.0', '70', 'US']
['Plymouth Fury iii', '14.0', '8', '440.0', '215.0', '4312.', '8.5', '70', 'US']
['Pontiac Catalina', '14.0', '8', '455.0', '225.0', '4425.', '10.0', '70', 'US']
['AMC Ambassador DPL', '15.0', '8', '390.0', '190.0', '3850.', '8.5', '70', 'US']


In [27]:
parser.close()

Got call to close generator
closing file


In [28]:
def parse_file(f_name):
    print('opening file...')
    f = open(f_name, 'r')
    try:
        dialect = csv.Sniffer().sniff(f.read(2000))
        f.seek(0)
        next(f) # skip header row
        reader = csv.reader(f, dialect=dialect)
        for row in reader:
            try:
                yield row
            except GeneratorExit:
                print('Got call to close generator')
                raise Exception('why, oh why did you do this?') from None
    finally:
        print('closing file')
        f.close()

In [29]:
parser = parse_file('cars.csv')
for row in itertools.islice(parser, 10):
    print(row)

opening file...
['Chevrolet Chevelle Malibu', '18.0', '8', '307.0', '130.0', '3504.', '12.0', '70', 'US']
['Buick Skylark 320', '15.0', '8', '350.0', '165.0', '3693.', '11.5', '70', 'US']
['Plymouth Satellite', '18.0', '8', '318.0', '150.0', '3436.', '11.0', '70', 'US']
['AMC Rebel SST', '16.0', '8', '304.0', '150.0', '3433.', '12.0', '70', 'US']
['Ford Torino', '17.0', '8', '302.0', '140.0', '3449.', '10.5', '70', 'US']
['Ford Galaxie 500', '15.0', '8', '429.0', '198.0', '4341.', '10.0', '70', 'US']
['Chevrolet Impala', '14.0', '8', '454.0', '220.0', '4354.', '9.0', '70', 'US']
['Plymouth Fury iii', '14.0', '8', '440.0', '215.0', '4312.', '8.5', '70', 'US']
['Pontiac Catalina', '14.0', '8', '455.0', '225.0', '4425.', '10.0', '70', 'US']
['AMC Ambassador DPL', '15.0', '8', '390.0', '190.0', '3850.', '8.5', '70', 'US']


In [30]:
parser.close()

Got call to close generator
closing file


Exception: why, oh why did you do this?

In [31]:
def save_to_db():
    print('starting new transaction')
    while True:
        try:
            data = yield
            print('sending data to datbase:', str(data))
        except GeneratorExit:
            print('committing transaction')
            raise

In [32]:
trans = save_to_db()

In [33]:
next(trans)

starting new transaction


In [34]:
trans.send('data 1')

sending data to datbase: data 1


In [35]:
trans.send('data 2')

sending data to datbase: data 2


In [36]:
trans.close()

committing transaction


In [46]:
def save_to_db():
    print('starting new transaction')
    while True:
        try:
            data = yield
            print('sending data to datbase:', eval(data))
        except Exception:
            print('aborting transaction')
        except GeneratorExit:
            print('committing transaction')
            raise

In [47]:
trans = save_to_db()

committing transaction


In [48]:
next(trans)

starting new transaction


In [49]:
trans.send('1 + 10')

sending data to datbase: 11


In [50]:
trans.send('1/0')

aborting transaction


In [51]:
from inspect import getgeneratorstate

In [52]:
getgeneratorstate(trans)

'GEN_SUSPENDED'

In [53]:
trans = save_to_db()

committing transaction


In [54]:
next(trans)

starting new transaction


In [55]:
trans.send('1 + 10')

sending data to datbase: 11


In [56]:
trans.close()

committing transaction


In [57]:
getgeneratorstate(trans)

'GEN_CLOSED'

In [58]:
def save_to_db():
    print('starting new transaction')
    is_abort = False
    try:
        while True:
            data = yield
            print('sending data to database:', eval(data))
    except Exception:
        is_abort = True
        raise
    finally:
        if is_abort:
            print('rollback transaction')
        else:
            print('commit transaction')

In [59]:
trans = save_to_db()
next(trans)
trans.send('1 + 1')
trans.close()

starting new transaction
sending data to database: 2
commit transaction


In [60]:
trans = save_to_db()
next(trans)
trans.send('1/0')
trans.close()

starting new transaction
rollback transaction


ZeroDivisionError: division by zero

In [61]:
class TransactionAborted(Exception):
    pass

def save_to_db():
    print('starting new transaction')
    is_abort = False
    try:
        while True:
            data = yield
            print('sending data to database:', eval(data))
    except Exception as ex:
        is_abort = True
        raise TransactionAborted(str(ex))
    finally:
        if is_abort:
            print('rollback transaction')
        else:
            print('commit transaction')

In [62]:
trans = save_to_db()
next(trans)
trans.send('1/0')
trans.close()

starting new transaction
rollback transaction


TransactionAborted: division by zero